Kaggle.com -> Python notebook -> SQL Server Database

## Extracting Data

In [1]:
# Download dataset from kaggle.com
# !pip install kaggle
import kaggle
!kaggle datasets download yasinnaal/bikes-sales-sample-data/

Dataset URL: https://www.kaggle.com/datasets/yasinnaal/bikes-sales-sample-data/versions/
License(s): CC0-1.0
bikes-sales-sample-data.zip: Skipping, found more recently modified local copy (use --force to force download)


In [2]:
# Downloaded data is in the zip file
# !pip install zipfile
import zipfile

with zipfile.ZipFile('bikes-sales-sample-data.zip', 'r') as zip_ref:
    # List all files in the ZIP archive
    files = zip_ref.namelist()
print(files)

['Addresses.csv', 'BusinessPartners.csv', 'Employees.csv', 'ProductCategories.csv', 'ProductCategoryText.csv', 'ProductTexts.csv', 'Products.csv', 'SalesOrderItems.csv', 'SalesOrders.csv']


In [3]:
# Extraxt files from zip file
zipfile_ref = zipfile.ZipFile('bikes-sales-sample-data.zip')
zipfile_ref.extractall() # Extract files to dir
zipfile_ref.close()

In [4]:
import os
import pandas as pd

# Path to the directory containing the files
current_dir = os.getcwd()

for file in files:
    # Full file path
    file_path = os.path.join(current_dir, file)
    
    df_name = file.rsplit('.', 1)[0]
    try:
        # Attempt to read the CSV file with different encodings
        df = pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        # If UTF-8 fails, try latin
        df = pd.read_csv(file_path, encoding='latin')
    except Exception as e:
        # Print a generic error message for other exceptions
        print(f"Error with file {file}: {e}")
    
    # Assign dataframe to global variable
    globals()[df_name] = df

## Transforming Data

In [5]:
# Function definitions for commonly used transformations

def lower_features(df):
    # Convert column names to lowercase
    df.columns = df.columns.str.lower()

def display_df(df):
    lower_features(df)
    print(f'DataFrame shape: {df.shape}')
    return df.head()


from datetime import datetime as dt 

# Convert date format from YYYYMMDD int64 to YYYY-MM-DD date
def convert_date(date_str):
    # Convert YYYYMMDD to datetime object
    date_obj = dt.strptime(str(date_str), '%Y%m%d')
    # Format to YYYY-MM-DD
    return date_obj.strftime('%Y-%m-%d')

def convert_to_datetime(df, columns):
    for col in columns:
        # Apply the date conversion
        df[col] = df[col].apply(convert_date)
        # Convert the column to datetime
        df[col] = pd.to_datetime(df[col], format='%Y-%m-%d')


# Convert date format for fiscalyearperion from YYYYMM int64 to YYYY-MM date
def convert_fiscal_year_period(fiscal):
    fiscal_str = str(fiscal)
    
    fiscal_year = fiscal_str[:4]
    fiscal_month = fiscal_str[-2:]
    return f'{fiscal_year}-{fiscal_month}'

def convert_fiscal(df, columns):
    for col in columns:
        df[col] = df[col].apply(convert_fiscal_year_period)
        df[col] = pd.to_datetime(df[col], format='%Y-%m')

# Validity dates
def validity_startdate(df, column):
    df[column] = pd.to_datetime('1900-01-01')
def validity_enddate(df, column):
    df[column] = pd.to_datetime('2100-12-31')

In [6]:
# Addresses DataFrame
display_df(Addresses)

DataFrame shape: (54, 12)


,addressid,city,postalcode,street,building,country,region,addresstype,validity_startdate,validity_enddate,latitude,longitude
0,1000000034,West Nyack,10994,Settlers Lane,5027.0,US,AMER,2,20000101,99991231,41.100057,-73.973562
1,1000000035,Fair Oaks,95628,Woodland Terrace,4467.0,US,AMER,2,20000101,99991231,38.638355,-121.286683
2,1000000036,Dunn,28334,Layman Avenue,2250.0,US,AMER,2,20000101,99991231,35.312013,-78.609950
3,1000000037,Chicago,60605,University Drive,4697.0,US,AMER,2,20000101,99991231,41.874591,-87.627303
4,1000000038,Ocala,34471,Bagwell Avenue,1565.0,US,AMER,2,20000101,99991231,29.183977,-82.118413


In [7]:
validity_startdate(Addresses, 'validity_startdate')
validity_enddate(Addresses, 'validity_enddate')

In [8]:
# Building number can't be decimal
Addresses['building'] =  pd.to_numeric(Addresses['building']).astype('Int64')

In [9]:
# Check data types
Addresses.dtypes

addressid                      int64
city                          object
postalcode                    object
street                        object
building                       Int64
country                       object
region                        object
addresstype                    int64
validity_startdate    datetime64[ns]
validity_enddate      datetime64[ns]
latitude                     float64
longitude                    float64
dtype: object

In [10]:
# BusinessPartners DataFrame
display_df(BusinessPartners)

DataFrame shape: (40, 14)


,partnerid,partnerrole,emailaddress,phonenumber,faxnumber,webaddress,addressid,companyname,legalform,createdby,createdat,changedby,changedat,currency
0,100000000,2,maria.brown@all4bikes.com,622734567,NaN,http://www.all4bikes.com,1000000034,All For Bikes,Inc.,10,20181003,10,20181003,USD
1,100000001,2,bob.buyer@amazebikes.com,3088530,NaN,http://www.amazebikes.com,1000000035,Amaze Bikes Inc,Inc.,13,20181003,13,20181003,USD
2,100000002,2,victor.sanchez@arenasportusa.com,3023352668,NaN,http://www.arenasportusa.com,1000000036,Arena Sports Inc,Inc.,14,20181003,14,20181003,USD
3,100000003,2,franklin.jones@atlantausa.com,511403266,NaN,http://www.atlantausa.com,1000000037,Atlanta Corp Inc,Inc.,10,20181003,10,20181003,USD
4,100000004,2,robert_brown@bikeworlds.com,2244668800,NaN,http://www.bikeworlds.com,1000000038,Bike World Inc,Inc.,4,20181003,4,20181003,USD


In [11]:
# Drop empty column
# BusinessPartners['phonenumber'].unique()
BusinessPartners = BusinessPartners.drop(columns=['faxnumber'])

In [12]:
# Convert date columns
convert_to_datetime(BusinessPartners, ['createdat', 'changedat'])

In [13]:
BusinessPartners.dtypes

partnerid                int64
partnerrole              int64
emailaddress            object
phonenumber              int64
webaddress              object
addressid                int64
companyname             object
legalform               object
createdby                int64
createdat       datetime64[ns]
changedby                int64
changedat       datetime64[ns]
currency                object
dtype: object

In [14]:
# Employees DataFrame
display_df(Employees)

DataFrame shape: (14, 19)


,employeeid,name_first,name_middle,name_last,name_initials,sex,language,phonenumber,emailaddress,loginname,addressid,validity_startdate,validity_enddate,unnamed: 13,unnamed: 14,unnamed: 15,unnamed: 16,unnamed: 17,unnamed: 18
0,1,Derrick,L,Magill,NaN,M,E,630-374-0306,derrick.magill@itelo.info,derrickm,1000000001,20000101,99991231,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Philipp,T,Egger,NaN,M,E,09603 61 24 64,philipp.egger@itelo.info,philippm,1000000002,20000101,99991231,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Ellis,K,Robertson,NaN,M,E,070 8691 2288,ellis.robertson@itelo.info,ellism,1000000003,20000101,99991231,NaN,NaN,NaN,NaN,NaN,NaN
3,4,William,M,Mussen,NaN,M,E,026734 4556,william.mussen@itelo.info,williamm,1000000004,20000101,99991231,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Javas,NaN,Hegde,NaN,M,E,02224135120,javas.hegde@itelo.info,javasm,1000000005,20000101,99991231,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
def create_full_name(row):
    # Create full_name column by combining first, last and, if exists, middle name
    if pd.isna(row['name_middle']):
        return f"{row['name_first']} {row['name_last']}"
    else:
        return f"{row['name_first']} {row['name_middle']} {row['name_last']}"

Employees['full_name'] = Employees.apply(create_full_name, axis=1)
# Insert new column to be the second one in the DataFrame 
Employees.insert(1, 'full_name', Employees.pop('full_name'))

In [16]:
# Drop redundant and empty columns
Employees.drop(columns=['name_initials', 'name_first', 'name_middle', 'name_last',
                        'unnamed: 13', 'unnamed: 14', 'unnamed: 15', 'unnamed: 16', 'unnamed: 17', 'unnamed: 18'],
              inplace=True)

In [17]:
validity_startdate(Employees, 'validity_startdate')
validity_enddate(Employees, 'validity_enddate')

In [18]:
# Format phone numbers
Employees['phonenumber'] = Employees['phonenumber'].str.replace('-', '')
Employees['phonenumber'] = Employees['phonenumber'].str.replace('.', '')
Employees['phonenumber'] = Employees['phonenumber'].str.replace(' ', '')

In [19]:
# Consolidate language with other tables
Employees['language'] = 'EN'

In [20]:
# Check the dataframe
Employees

,employeeid,full_name,sex,language,phonenumber,emailaddress,loginname,addressid,validity_startdate,validity_enddate
0,1,Derrick L Magill,M,EN,6303740306,derrick.magill@itelo.info,derrickm,1000000001,1900-01-01,2100-12-31
1,2,Philipp T Egger,M,EN,09603612464,philipp.egger@itelo.info,philippm,1000000002,1900-01-01,2100-12-31
2,3,Ellis K Robertson,M,EN,07086912288,ellis.robertson@itelo.info,ellism,1000000003,1900-01-01,2100-12-31
3,4,William M Mussen,M,EN,0267344556,william.mussen@itelo.info,williamm,1000000004,1900-01-01,2100-12-31
4,5,Javas Hegde,M,EN,02224135120,javas.hegde@itelo.info,javasm,1000000005,1900-01-01,2100-12-31
5,6,Haseena al Yousuf,F,EN,2288340,alhaseena.yousuf@itelo.info,haseena alf,1000000006,1900-01-01,2100-12-31
6,7,Roberta M Holloway,F,EN,3065987404,roberta.holloway@itelo.info,robertaf,1000000007,1900-01-01,2100-12-31
7,8,Pénélope G Duperré,F,EN,03234610404,penelope.duperre@itelo.info,penelopef,1000000008,1900-01-01,2100-12-31
8,9,Kirk J Lee,M,EN,8057566064,kirk.lee@itelo.info,kirkm,1000000009,1900-01-01,2100-12-31
9,10,Janet R Gray,F,EN,2018492465,janet.gray@itelo.info,janetf,1000000010,1900-01-01,2100-12-31


In [21]:
# ProductCategories DataFrame
display_df(ProductCategories)

DataFrame shape: (9, 3)


,prodcategoryid,createdby,createdat
0,RO,12,20181003
1,BX,4,20181003
2,CC,7,20181003
3,MB,11,20181003
4,RC,9,20181003


In [22]:
# Convert date column
convert_to_datetime(ProductCategories, ['createdat'])

In [23]:
# ProductCategoryText DataFrame
display_df(ProductCategoryText)

DataFrame shape: (9, 5)


,prodcategoryid,language,short_descr,medium_descr,long_descr
0,RO,EN,Road Bike,NaN,NaN
1,BX,EN,BMX,NaN,NaN
2,CC,EN,Cyclo-cross Bike,NaN,NaN
3,MB,EN,Mountain Bike,NaN,NaN
4,RC,EN,Racing Bike,NaN,NaN


In [24]:
# Drop empty columns
ProductCategoryText.drop(columns=['medium_descr', 'long_descr'], inplace=True)

In [25]:
# Products DataFrame
display_df(Products)

DataFrame shape: (42, 19)


,productid,typecode,prodcategoryid,createdby,createdat,changedby,changedat,supplier_partnerid,taxtariffcode,quantityunit,weightmeasure,weightunit,currency,price,width,depth,height,dimensionunit,productpicurl
0,RO-1001,PR,RO,9,20181003,9,20181003,100000000,1,EA,7.7,KG,USD,525,NaN,NaN,NaN,NaN,NaN
1,RO-1002,PR,RO,9,20181003,9,20181003,100000001,1,EA,8.0,KG,USD,689,NaN,NaN,NaN,NaN,NaN
2,RO-1003,PR,RO,12,20181003,12,20181003,100000002,1,EA,9.1,KG,USD,721,NaN,NaN,NaN,NaN,NaN
3,BX-1011,PR,BX,9,20181003,9,20181003,100000003,1,EA,11.1,KG,USD,249,NaN,NaN,NaN,NaN,NaN
4,BX-1012,PR,BX,6,20181003,6,20181003,100000004,1,EA,12.0,KG,USD,399,NaN,NaN,NaN,NaN,NaN


In [26]:
# Rename columns
Products = Products.rename(columns={'weightmeasure': 'weight(kg)', 'price': 'price(usd)'})

In [27]:
# Drop redundant and empty columns
Products.drop(columns=['typecode', 'weightunit', 'currency', 'width',
                       'depth', 'height','dimensionunit', 'productpicurl']
              , inplace=True)

In [28]:
# Change value to self-explanator
Products['quantityunit'] = Products['quantityunit'].str.replace('EA', 'each')

In [29]:
# Convert date columns
convert_to_datetime(Products, ['createdat', 'changedat'])

In [30]:
# ProductTexts DataFrame
display_df(ProductTexts)

DataFrame shape: (44, 5)


,productid,language,short_descr,medium_descr,long_descr
0,RO-1001,EN,Roady 1001,NaN,NaN
1,RO-1002,EN,Roady 1002,NaN,NaN
2,RO-1003,EN,Roady 1003,NaN,NaN
3,BX-1011,EN,BMX Vintage 1011,NaN,NaN
4,BX-1012,EN,BMX Jump 1012,NaN,NaN


In [31]:
# Drop empty column
# ProductTexts['long_descr'].unique() # all NaN
ProductTexts.drop(columns=['long_descr'], inplace=True)

In [32]:
# SalesOrderItems DataFrame
display_df(SalesOrderItems)

DataFrame shape: (1930, 13)


,salesorderid,salesorderitem,productid,noteid,currency,grossamount,netamount,taxamount,itematpstatus,opitempos,quantity,quantityunit,deliverydate
0,500000000,10,MB-1034,,USD,2499,2186.625,312.375,I,NaN,4,EA,20180311
1,500000000,20,CB-1161,,USD,399,349.125,49.875,I,NaN,9,EA,20180311
2,500000001,10,HB-1175,,USD,899,786.625,112.375,I,NaN,2,EA,20180228
3,500000001,20,RC-1056,,USD,2499,2186.625,312.375,I,NaN,2,EA,20180228
4,500000001,30,CC-1021,,USD,1144,1001.000,143.000,I,NaN,3,EA,20180228


In [33]:
SalesOrderItems.dtypes

salesorderid        int64
salesorderitem      int64
productid          object
noteid             object
currency           object
grossamount         int64
netamount         float64
taxamount         float64
itematpstatus      object
opitempos         float64
quantity            int64
quantityunit       object
deliverydate        int64
dtype: object

In [34]:
# Change value to self-explanator
SalesOrderItems['quantityunit'] = SalesOrderItems['quantityunit'].str.replace('EA', 'each')

In [35]:
# Drop empty columns
SalesOrderItems.drop(columns=['noteid','itematpstatus', 'opitempos'], inplace=True)

In [36]:
# Correct invalid data values
SalesOrderItems['deliverydate'] = SalesOrderItems['deliverydate'].replace(29991212, 20191212)
# Convert date column
convert_to_datetime(SalesOrderItems, ['deliverydate'])
# Change column type
SalesOrderItems['deliverydate'] = pd.to_datetime(SalesOrderItems['deliverydate']) 

In [37]:
# SalesOrders DataFrame
display_df(SalesOrders)

DataFrame shape: (334, 17)


,salesorderid,createdby,createdat,changedby,changedat,fiscvariant,fiscalyearperiod,noteid,partnerid,salesorg,currency,grossamount,netamount,taxamount,lifecyclestatus,billingstatus,deliverystatus
0,500000000,4,20180111,4,20180116,K4,2018001,NaN,100000022,APJ,USD,13587,11888.625,1698.375,C,C,C
1,500000001,2,20180112,2,20180115,K4,2018001,NaN,100000026,EMEA,USD,12622,11044.250,1577.750,C,C,C
2,500000002,5,20180115,5,20180120,K4,2018001,NaN,100000018,APJ,USD,45655,39948.125,5706.875,C,C,C
3,500000003,3,20180115,3,20180120,K4,2018001,NaN,100000009,EMEA,USD,101786,89062.750,12723.250,C,C,C
4,500000004,8,20180116,8,20180117,K4,2018001,NaN,100000025,EMEA,USD,71684,62723.500,8960.500,C,C,C


In [38]:
# Drop empty and redundant column
# SalesOrders['noteid'].unique()
SalesOrders.drop(columns=['noteid', 'fiscvariant'], inplace=True)

In [39]:
# Convert date columns
convert_to_datetime(SalesOrders, ['createdat', 'changedat'])
# Convert fiscalyearperiod
convert_fiscal(SalesOrders, ['fiscalyearperiod'])

In [40]:
print(SalesOrders.lifecyclestatus.unique())
print(SalesOrders.billingstatus.unique())
print(SalesOrders.deliverystatus.unique())

['C' 'X' 'I']
['C' 'X' 'I']
['C' 'X' 'I']


In [41]:
# Change statuses to self-explanatory values
statuses = ['lifecyclestatus', 'billingstatus', 'deliverystatus']
for col in statuses:
    SalesOrders[col] = SalesOrders[col].str.replace('C', 'Completed')
    SalesOrders[col] = SalesOrders[col].str.replace('X', 'Cancelled')
    SalesOrders[col] = SalesOrders[col].str.replace('I', 'Initiated')

## Load Data
### To SQL Server Database

In [83]:
import pandas as pd
from sqlalchemy import create_engine

# Create a connection engine
engine = create_engine(f'mssql+pyodbc://ISIDORA\\MSSQLSERVER/SAPBikesSales?driver=ODBC+Driver+18+for+SQL+Server')

# Dictionary of DataFrames
dfs = {
    'Addresses': Addresses,
    'BusinessPartners': BusinessPartners,
    'Employees': Employees,
    'ProductCategories': ProductCategories,
    'ProductCategoryText': ProductCategoryText,
    'ProductTexts': ProductTexts,
    'Products': Products,
    'SalesOrderItems': SalesOrderItems,
    'SalesOrders': SalesOrders
}
# Upload dataframes to sql server
for table_name, df in dfs.items():
    try:
        df.to_sql(table_name, con=engine, if_exists='append', index=False)
        print(f'Successfully uploaded {table_name} to SQL Server.')
    except Exception as e:
        print(f'Error uploading {table_name}: {e}')

Successfully uploaded Addresses to SQL Server.
Successfully uploaded BusinessPartners to SQL Server.
Successfully uploaded Employees to SQL Server.
Successfully uploaded ProductCategories to SQL Server.
Successfully uploaded ProductCategoryText to SQL Server.
Successfully uploaded ProductTexts to SQL Server.
Successfully uploaded Products to SQL Server.
Successfully uploaded SalesOrderItems to SQL Server.
Successfully uploaded SalesOrders to SQL Server.
